In [1]:
import torch
from torch import nn
import data_utils
from training.MADGAN_train import MadGanTrainingPipeline
from models.MADGAN import Generator, Discriminator, AnomalyDetector
from utils import evaluation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Parameters

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [3]:
model_type = "MAD-GAN"
num_generated_features = 7
seq_length = 30
seq_stride = 10

random_seed = 0
num_epochs = 100
batch_size = 8
lr = 1e-5
wd = 5e-7
latent_dim = 500
hidden_dim = 500

# Load data

In [4]:
#train_dl, test_dl = data_utils.kdd99(seq_length, seq_stride, num_generated_features,seq_length,batch_size)

In [5]:
file_path = './data/Stocks/aapl.us.txt'
tscv_dl_list = data_utils.load_stock_as_crossvalidated_timeseries(file_path, seq_length, seq_stride, seq_length, batch_size)

# Model

In [7]:
pipeline = MadGanTrainingPipeline()

In [8]:
generator = Generator(
    latent_space_dim=latent_dim,
    hidden_units=hidden_dim,
    output_dim=num_generated_features)
generator.to(device=DEVICE)

Generator(
  (lstm): LSTM(500, 500, num_layers=2, batch_first=True, dropout=0.1)
  (linear): Linear(in_features=500, out_features=7, bias=True)
)

In [9]:
discriminator = Discriminator(input_dim=num_generated_features,
    hidden_units=hidden_dim,
    add_batch_mean=False)
discriminator.to(device=DEVICE)

Discriminator(
  (lstm): LSTM(7, 500, num_layers=2, batch_first=True, dropout=0.1)
  (linear): Linear(in_features=500, out_features=1, bias=True)
  (activation): Sigmoid()
)

# Loss and Optimizer

In [10]:
def loss_function(inputs, targets):
    return nn.BCELoss()(inputs, targets)

In [11]:
discriminator_optim = torch.optim.Adam(discriminator.parameters(), lr=lr, weight_decay=wd)
generator_optim = torch.optim.Adam(generator.parameters(), lr=lr, weight_decay=wd)

# Train

In [12]:
pipeline.train(seq_length, latent_dim, tscv_dl_list, discriminator, generator, discriminator_optim, generator_optim, loss_function, random_seed, num_epochs, DEVICE)

Epoch 0 training:
G_loss: 0.17595690488815308, D_loss_real: 0.17520183622837066, D_loss_fake: 1.8816189289093017
Epoch 1 training:
G_loss: 0.22949271500110627, D_loss_real: 0.1929372876882553, D_loss_fake: 1.6361552476882935
Epoch 2 training:
G_loss: 0.2953533411026001, D_loss_real: 0.21076650321483612, D_loss_fake: 1.4159012317657471
Epoch 3 training:
G_loss: 0.3774519443511963, D_loss_real: 0.22913762629032136, D_loss_fake: 1.2137305736541748
Epoch 4 training:
G_loss: 0.46087435483932493, D_loss_real: 0.248876091837883, D_loss_fake: 1.0498417854309081
Evaluation metrics: {'D_loss': 1.2271162271499634, 'G_acc': 0.9854166954755783, 'D_acc': 0.5072916895151138}
EMMV evaluation:
0.0001 1044813.3782958984
Epoch 0 training:
G_loss: 0.5659577515390184, D_loss_real: 0.26947342024909127, D_loss_fake: 0.8938878509733412
Epoch 1 training:
G_loss: 0.7015186150868734, D_loss_real: 0.29899924993515015, D_loss_fake: 0.7411307560073005
Epoch 2 training:
G_loss: 0.7839267585012648, D_loss_real: 0.324

KeyboardInterrupt: 